<a href="https://colab.research.google.com/github/mirovix/CV_final_project/blob/main/NN_boats_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary packages
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from imutils.object_detection import non_max_suppression
import numpy as np
import argparse
import cv2
from google.colab.patches import cv2_imshow
from google.colab import drive
from google.colab import files
import json
import io
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


np.random.seed(42)

In [ ]:
def get_iou(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [ ]:
def plot_loss(history):
  plt.figure(figsize=(10,6))
  plt.plot(history.epoch,history.history['loss'])
  plt.plot(history.epoch,history.history['val_loss'])
  plt.title('loss')

def plot_accuracy(history):
  plt.figure(figsize=(10,6))
  plt.plot(history.epoch,history.history['accuracy'])
  plt.plot(history.epoch,history.history['val_accuracy'])
  plt.title('accuracy')

In [ ]:
def non_max_suppression_slow(boxes, overlapThresh):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
	# initialize the list of picked indexes
	pick = []
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
  # keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list, add the index
		# value to the list of picked indexes, then initialize
		# the suppression list (i.e. indexes that will be deleted)
		# using the last index
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		suppress = [last]
    		# loop over all indexes in the indexes list
		for pos in range(0, last):
			# grab the current index
			j = idxs[pos]
			# find the largest (x, y) coordinates for the start of
			# the bounding box and the smallest (x, y) coordinates
			# for the end of the bounding box
			xx1 = max(x1[i], x1[j])
			yy1 = max(y1[i], y1[j])
			xx2 = min(x2[i], x2[j])
			yy2 = min(y2[i], y2[j])
			# compute the width and height of the bounding box
			w = max(0, xx2 - xx1 + 1)
			h = max(0, yy2 - yy1 + 1)
			# compute the ratio of overlap between the computed
			# bounding box and the bounding box in the area list
			overlap = float(w * h) / area[j]
			# if there is sufficient overlap, suppress the
			# current bounding box
			if overlap > overlapThresh:
				suppress.append(pos)
		# delete all indexes from the index list that are in the
		# suppression list
		idxs = np.delete(idxs, suppress)
	# return only the bounding boxes that were picked
	return boxes[pick]

In [ ]:
drive.mount('/content/drive')

with open('/content/drive/MyDrive/dataset_boats/label_multi.json') as json_file:
    json_file = json.load(json_file)

i=1
train_images=[]
train_labels=[]

for row in json_file:
  if(i==1433):
    break;
  if(i%20==0):
    print(i)
  i=i+1
  gtvalues=[]
  for col in row['Label']['objects']:
    if(len(row['Label']) != 0):
      x_start = int(col['bbox']['left'])
      y_start = int(col['bbox']['top'])
      x_end = int(col['bbox']['left'] + col['bbox']['width'])
      y_end = int(col['bbox']['top'] + col['bbox']['height'])
      gtvalues.append({"x1":x_start,"x2":x_end,"y1":y_start,"y2":y_end})

  image_path = "/content/drive/MyDrive/dataset_boats/train/" + row['External ID']
  cv_image = cv2.imread(image_path)

  ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
  ss.setBaseImage(cv_image)
  ss.switchToSelectiveSearchFast()
  ssresults = ss.process()

  imout = cv_image.copy()
  counter = 0
  falsecounter = 0
  flag = 0
  fflag = 0
  bflag = 0
  for e,result in enumerate(ssresults):
    if e < 2000 and flag == 0:
      for gtval in gtvalues:

        x, y, w, h = result
        iou = get_iou(gtval,{"x1":x,"x2":x+w,"y1":y,"y2":y+h})

        if counter < 2:
          if iou > 0.95:
            resized = cv2.resize(imout[y:y+h,x:x+w], (224,224), interpolation = cv2.INTER_AREA)
            train_images.append(resized)
            train_labels.append(1)
            counter += 1
        else :
          fflag =1
        if falsecounter < 5:
          if h*w < 3000:
            continue
          if iou < 0.1:
            resized = cv2.resize(imout[y:y+h,x:x+w], (224,224), interpolation = cv2.INTER_AREA)
            train_images.append(resized)
            train_labels.append(0)
            falsecounter += 1
        else :
          bflag = 1
      if fflag == 1 and bflag == 1:
        flag = 1

#print(gtvalues) 
print(len(train_images))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980


In [ ]:
from imutils import paths
imagePaths = sorted(list(paths.list_images("/content/drive/MyDrive/dataset_boats/buildings/"))) 
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
for imagePath in imagePaths:
  image = cv2.imread(imagePath)
  imout = image.copy()
  ss.setBaseImage(image)
  ss.switchToSelectiveSearchFast()
  ssresults = ss.process()
  flag=0
  falsecounter = 0
  for e,result in enumerate(ssresults):
    if e < 2000 and flag == 0:
      x, y, w, h = result
      if h*w < 3000:
        continue
      if falsecounter < 3:
        resized = cv2.resize(imout[y:y+h,x:x+w], (224,224), interpolation = cv2.INTER_AREA)
        train_images.append(resized)
        train_labels.append(0)
        falsecounter += 1
      else :
        flag = 1
print(len(train_images))

In [ ]:
X_new = np.array(train_images)
y_new = np.array(train_labels)

from keras.applications.xception import Xception
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import Dense
from keras import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

vggmodel = MobileNetV2(weights='imagenet', include_top=True)

X=vggmodel.layers[-2].output
predictions = Dense(2, activation="softmax")(X)
model_final = Model(inputs = vggmodel.input, outputs = predictions)
opt = Adam(lr=0.0001)
model_final.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])
model_final.summary()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
class MyLabelBinarizer(LabelBinarizer):
    def transform(self, y):
        Y = super().transform(y)
        if self.y_type_ == 'binary':
            return np.hstack((Y, 1-Y))
        else:
            return Y
    def inverse_transform(self, Y, threshold=None):
        if self.y_type_ == 'binary':
            return super().inverse_transform(Y[:, 0], threshold)
        else:
            return super().inverse_transform(Y, threshold)
lenc = MyLabelBinarizer()
Y =  lenc.fit_transform(y_new)
#X_train, X_test , y_train, y_test = train_test_split(X_new,Y,test_size=0.10)
X_train, X_test, y_train, y_test = train_test_split(X_new,Y,test_size=0.10)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.12)

In [ ]:
#mean = np.mean(X_train, axis = 0)
#std = np.std(X_train, axis = 0)

#X_train = (X_train - mean) /  std
#X_val = (X_val - mean) /  std
#X_test = (X_test - mean) /  std

trdata = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
X_train = trdata.flow(x=X_train, y=y_train)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
#checkpoint = ModelCheckpoint("ieeercnn_vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')
#history = model_final.fit_generator(generator=X_train, steps_per_epoch= 10, epochs= 5, validation_data= X_val, validation_steps=2)
history = model_final.fit(X_train , epochs=10, batch_size=16 ,validation_data=(X_val, y_val))

In [ ]:
plot_loss(history)
plot_accuracy(history)

scores = model_final.evaluate(X_test, y_test, verbose=2)
print("%s: %.2f%%" % (model_final.metrics_names[1], scores[1]*100))

In [ ]:
image_path = "/content/drive/MyDrive/dataset_boats/train/20130412_150512_31795.jpg" #20130412_151959_34446 #"/content/drive/MyDrive/dataset_boats/venice_dataset/07.png" 
image_path = "/content/drive/MyDrive/dataset_boats/venice_dataset/10.png"
#image_path = "/content/drive/MyDrive/dataset_boats/Kaggle_ships/01.jpg" 
#image_path = "/content/drive/MyDrive/dataset_boats/other_boats/a.jpg"
# speed-up using multithreads

import tensorflow as tf
model_final = tf.keras.models.load_model("/content/drive/MyDrive/dataset_boats/model7.h5")
cv2.setUseOptimized(True)  # Use optimization
#cv2.setNumThreads(8)  # Turn on multithreaded computing
img = cv2.imread(image_path)
imout = img.copy()
cv2.resize(img, (8,8), interpolation = cv2.INTER_AREA)
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(img)
ss.switchToSingleStrategy()
#ss.switchToSelectiveSearchFast(10,10,0.001)
ssresults = ss.process()
boundingBoxes = []
score = []
for i,result in enumerate(ssresults):
  if(i>3000):
    break
  if(i%250==0):
    print(i)
  x,y,w,h = result
  if ((w*h)<5000) or (h > 220):
    continue
  timage = imout[y:y+h,x:x+w]
  resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
  img = np.expand_dims(resized, axis=0)
  out= model_final.predict(img)
  if out[0][0] > 0.95:
    #cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 255, 255), 1, cv2.LINE_AA)
    cv2_imshow(resized)
    boundingBoxes.append([x, y, x+w, y+h])
    #score.append(out[0][0])

boundingBoxes = np.array(boundingBoxes)
pick = non_max_suppression_slow(boundingBoxes, 0.3)
#cv2.NMSBoxes(boundingBoxes, score, 0.3, 0.1)
for (x, y, x_2, y_3) in pick:
   cv2.rectangle(imout, (x, y), (x_2, y_3), (0, 255, 0), 1, cv2.LINE_AA)

cv2_imshow(imout)

In [ ]:
model_final.save("/content/drive/MyDrive/dataset_boats/model8.h5", overwrite=True)

In [ ]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2 
import tensorflow as tf
model = tf.keras.models.load_model('/content/drive/MyDrive/dataset_boats/model6.h5')
full_model = tf.function(lambda inputs: model(inputs))    
full_model = full_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
# Get frozen ConcreteFunction    
frozen_func = convert_variables_to_constants_v2(full_model)    
frozen_func.graph.as_graph_def()
# Save frozen graph from frozen ConcreteFunction to hard drive    
tf.io.write_graph(graph_or_graph_def=frozen_func.graph, logdir="/content/drive/MyDrive/dataset_boats/", name="model.pb", as_text=False)
#tf.io.write_graph(graph_or_graph_def=frozen_func.graph, logdir="/content/drive/MyDrive/dataset_boats/", name="model.pbtxt", as_text=True)

'/content/drive/MyDrive/dataset_boats/model.pb'

In [ ]:
from keras import backend as K
with open('/content/drive/MyDrive/dataset_boats/model.pb', 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

with K.get_session() as sess:
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')

for i in reversed(range(len(graph_def.node))):
    if graph_def.node[i].op == 'Const':
        del graph_def.node[i]
    for attr in ['T', 'data_format', 'Tshape', 'N', 'Tidx', 'Tdim','use_cudnn_on_gpu', 'Index', 'Tperm', 'is_training','Tpaddings']:
        if attr in graph_def.node[i].attr:
             del graph_def.node[i].attr[attr]

# Save stripped model.
tf.io.write_graph(graph_def, logdir="/content/drive/MyDrive/dataset_boats/", name="model.pbtxt", as_text=True)

'/content/drive/MyDrive/dataset_boats/model.pbtxt'

In [ ]:
# Architecture and weight files for the model
textGraph = "/content/drive/MyDrive/dataset_boats/model.pbtxt"
modelWeights = "/content/drive/MyDrive/dataset_boats/model.pb"
# Load the network
net = cv2.dnn.readNetFromTensorflow(modelWeights, textGraph)

In [ ]:
!pip install opencv-python==4.5.1.48
!pip install opencv-contrib-python==4.5.1.48

In [ ]:
image_path = "/content/drive/MyDrive/dataset_boats/venice_dataset/07.png"
image_path = "/content/drive/MyDrive/dataset_boats/Kaggle_ships/10.jpg"
img = cv2.imread(image_path)


ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(img)
ss.switchToSingleStrategy()
#ss.switchToSelectiveSearchFast()
ssresults = ss.process()
imout = img.copy()
boundingBoxes = []
for i,result in enumerate(ssresults):
  if(i>2000):
    break
  if(i%250==0):
    print(i)
  x,y,w,h = result
  timage = imout[y:y+h,x:x+w]
  resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
  img = cv2.dnn.blobFromImage(resized, size=(224, 224), swapRB=False, crop=False)
  net.setInput(img)
  out = net.forward()
  if out[0][0] > 0.60:
    #cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
    boundingBoxes.append([x, y, x+w, y+h])

boundingBoxes = np.array(boundingBoxes)
pick = non_max_suppression_slow(boundingBoxes, 0.1)
for (x, y, x_2, y_3) in pick:
   cv2.rectangle(imout, (x, y), (x_2, y_3), (0, 255, 0), 1, cv2.LINE_AA)
cv2_imshow(imout)

0
250
500
750
1000


In [ ]:
print(boundingBoxes)